# Nearest airport request performance

In [1]:
import owlready2 as owl
import pandas as pd
import numpy as np

In [2]:
onto_filename = "../../src/ontology/final-archi-individuals.owl"
onto = owl.get_ontology(onto_filename).load()

## Full SPARQL

In [3]:
def test_full_sparql():
    # Returns (Name, ICAO, L1-distance) (no complex functions in SPARQL...)
    lat, lng = 43.56759421172615, 1.458735401391755
    l = list(owl.default_world.sparql(
        f"""
            PREFIX pie:<http://www.semanticweb.org/clement/ontologies/2020/1/final-archi#>
            SELECT ?name ?ICAO ?latitude ?longitude ?l1_dist
            WHERE {{
                BIND({lat} AS ?plane_lat)
                BIND({lng} AS ?plane_lng)
                BIND(ABS(?latitude-?plane_lat) AS ?diff_lat) .
                BIND(ABS(?longitude-?plane_lng) AS ?diff_lng) .
                BIND((?diff_lat+?diff_lng) AS ?l1_dist) .
                ?Airport pie:AirportICAOCode ?ICAO .
                ?Airport pie:AirportName ?name .
                ?Airport pie:AirportGPSLatitude ?latitude .
                ?Airport pie:AirportGPSLongitude ?longitude .
            }}
            ORDER BY ?l1_dist LIMIT 1
        """))
    return tuple(l[0])

In [4]:
test_full_sparql()

('Toulouse Blagnac Airport', 'LFBO', 43.634998, 1.367778, 0.15836118966560542)

## With Pandas

In [13]:
def coord_to_dist(cur_lat, cur_long, dest_lat, dest_long):
    cur_lat = cur_lat*np.pi/180
    cur_long = cur_long*np.pi/180
    dest_lat = dest_lat*np.pi/180
    dest_long = dest_long*np.pi/180
    return 60*180/np.pi*np.arccos(np.sin(cur_lat)*np.sin(dest_lat)+np.cos(cur_lat)*np.cos(dest_lat)*np.cos(dest_long-cur_long))

def test_pandas():
    # Returns ()
    lat, lng = 43.56759421172615, 1.458735401391755
    response = list(owl.default_world.sparql(
        f"""
        PREFIX pie:<http://www.semanticweb.org/clement/ontologies/2020/1/final-archi#>
        SELECT ?Name ?ICAO ?lat ?long
        WHERE {{
            ?Airport pie:AirportICAOCode ?ICAO .
            ?Airport pie:AirportName ?Name .
            ?Airport pie:AirportGPSLongitude ?long .
            ?Airport pie:AirportGPSLatitude ?lat .
            }}
        """))

    l = np.array(response)

    df=  pd.DataFrame({
        'name': l[:, 0],
        'ICAO': l[:, 1],
        'lat': l[:, 2],
        'lng': l[:, 3],
    })

    df['lat'] = pd.to_numeric(df['lat'], downcast="float")
    df['lng'] = pd.to_numeric(df['lng'], downcast="float")

    df['distance'] = df.apply(lambda x: coord_to_dist(x["lat"], x["lng"], lat, lng), axis=1)
    return df.iloc[df['distance'].idxmin()].to_dict()

In [14]:
test_pandas()

{'name': 'Toulouse Blagnac Airport',
 'ICAO': 'LFBO',
 'lat': 43.635,
 'lng': 1.367778,
 'distance': 5.654606958403692}

## Database + Pandas

In [7]:
from models import Airport

In [8]:
def test_db():
    lat, lng = 43.56759421172615, 1.458735401391755
    data = Airport.query.with_entities(Airport.name, Airport.icao, Airport.latitude, Airport.longitude).all()
    df = pd.DataFrame(data, columns=['name', 'ICAO', 'lat', 'lng'])
    df['lat'] = pd.to_numeric(df['lat'], downcast="float")
    df['lng'] = pd.to_numeric(df['lng'], downcast="float")

    df['distance'] = df.apply(lambda x: coord_to_dist(x["lat"], x["lng"], lat, lng), axis=1)
    return tuple(df.iloc[df['distance'].idxmin()])

In [9]:
test_db()

('Toulouse Blagnac Airport', 'LFBO', 43.635, 1.367778, 5.654606958403692)

## Compare performance

In [10]:
%%timeit
test_full_sparql()

1.95 s ± 159 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
test_pandas()

400 ms ± 119 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
test_db()

The slowest run took 7.09 times longer than the fastest. This could mean that an intermediate result is being cached.
385 ms ± 301 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
